Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

### Read in data and create dataframe

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [240]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Engineer at least two new features. (See below for explanation & ideas.)

In [241]:
# does the apartements allow pets? 
# If dogs or cats are allowed then we want a 1 otherwise we want 0

df.loc[(df['cats_allowed'] == 1) | (df['dogs_allowed'] == 1), 'pets_allowed'] = 1
df.loc[(df['cats_allowed'] == 0) | (df['dogs_allowed'] == 1), 'pets_allowed'] = 1
df.loc[(df['cats_allowed'] == 1) | (df['dogs_allowed'] == 0), 'pets_allowed'] = 1

print((df['pets_allowed'] == 1).value_counts())
df.head()



True    48817
Name: pets_allowed, dtype: int64


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0


In [242]:
# luxury apartments... this is a little arbitrary but we will used certain features
# as those that would be considered luxury

# has doorman, fitness_center, terrace, exclusive, garden patio, roof_deck
# sum to find how many luxury features each apartment has.

luxury_features = ['doorman', 'fitness_center', 'terrace', 'exclusive', 'garden_patio', 'roof_deck']

df['luxury_features'] = df[luxury_features].sum(axis=1)

print(df['luxury_features'].value_counts())
df.head()

0    24174
1    10177
2     8316
3     5011
4      922
5      217
Name: luxury_features, dtype: int64


,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed,luxury_features
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0


In [243]:
# lets also make the interest level into a 1-3 scale. 
# 1 for low, 2 for medium, 3 for high

df['interest_level'].value_counts()

low       33946
medium    11181
high       3690
Name: interest_level, dtype: int64

In [244]:
df['interest_level'] = df['interest_level'].astype('category')


df['interest_level'] = df['interest_level'].cat.codes

df['interest_level'].value_counts()

1    33946
2    11181
0     3690
Name: interest_level, dtype: int64

### Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.


In [245]:
# create train data only data from April & May of 2016

train_start_date = '2016-04-01'
train_end_date = '2016-05-31'
test_start_date = '2016-06-01'
test_end_date = '2016-06-30'


train = df[(df['created'] >= train_start_date) & (df['created'] <= train_end_date)]
test = df[(df['created'] >= test_start_date) & (df['created'] <= test_end_date)]

print("dimensions of training data", train.shape)
print("dimensions of test data", test.shape)
print("dimensions of entire data", df.shape)

print('test data is', test.shape[0]/df.shape[0] *100, "% of the dataset")
print('training data is', train.shape[0]/df.shape[0] *100, "% of the dataset")

dimensions of training data (31839, 36)
dimensions of test data (16973, 36)
dimensions of entire data (48817, 36)
test data is 34.76862568367577 % of the dataset
training data is 65.22113198271094 % of the dataset


In [246]:
train.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed,luxury_features
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
5,2.0,4,2016-04-19 04:24:47,,West 18th Street,40.7429,-74.0028,7995,350 West 18th Street,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
6,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,40.8012,-73.9660,3600,210 West 107th Street,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0


In [247]:
test.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,pets_allowed,luxury_features
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,2
11,1.0,1,2016-06-03 03:21:22,Check out this one bedroom apartment in a grea...,W. 173rd Street,40.8448,-73.9396,1675,644 W. 173rd Street,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
14,1.0,1,2016-06-01 03:11:01,Spacious 1-Bedroom to fit King-sized bed comfo...,East 56th St..,40.7584,-73.9648,3050,315 East 56th St..,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0
24,2.0,4,2016-06-07 04:39:56,SPRAWLING 2 BEDROOM FOUND! ENJOY THE LUXURY OF...,W 18 St.,40.7391,-73.9936,7400,30 W 18 St.,2,1,1,1,1,1,1,0,0,1,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,1.0,3


### Fit a linear regression model with at least two features

In [0]:
#import linear regression from sklearn

from sklearn.linear_model import LinearRegression

In [0]:
# set model class, instatiate linear regression

model = LinearRegression(fit_intercept=True)

In [0]:
# set the x features matrix and the y target matrix vectors

features = ['interest_level', 'pets_allowed', 'luxury_features']
target = ['price']

X_train = train[features]
X_test = test[features]
y_train = train[target]

In [251]:
# Fit the model

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [252]:
# apply model to the test data

y_pred = model.predict(X_test)
y_pred.T

array([[3049.81622269, 4043.35709784, 3182.78739622, ..., 3049.81622269,
        3613.07224703, 3049.81622269]])

### Get the model's coefficients and intercept.

In [253]:
# Get the model's coefficients and intercept.

print('The intercept for the linear regression model is', model.intercept_[0])

pd.DataFrame(model.coef_.T, index=X_train.columns, columns=['Coefficients'])

The intercept for the linear regression model is 3315.7585697543323


,Coefficients
interest_level,-132.971174
pets_allowed,0.000000
luxury_features,430.284851


### Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.

In [0]:
import numpy as np
from numpy import ndarray
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [255]:
np.sqrt(mean_squared_error(test['price'], y_pred))

1682.6976219473624

In [256]:
mean_absolute_error(test['price'], y_pred)

1137.008173060534

In [257]:
r2_score(test['price'], y_pred)

0.08897943646837758

In [258]:
## Lets check for the train dataset

y_pred = model.predict(X_train)
y_pred

array([[3315.75856975],
       [3182.78739622],
       [3182.78739622],
       ...,
       [3613.07224703],
       [3613.07224703],
       [3182.78739622]])

In [259]:
np.sqrt(mean_squared_error(y_train, y_pred))

1690.0974627762596

In [260]:
mean_absolute_error(y_train, y_pred)

1150.5763555213925

In [261]:
r2_score(y_train, y_pred)

0.08016400930731349

### try and feature engineer some new features to get a better outcome

In [317]:
!pip install uszipcode

In [0]:
from uszipcode import Zipcode
from uszipcode import SearchEngine

In [0]:

search = SearchEngine(simple_zipcode=True)
zipcode,  = search.by_coordinates(lat = 40.7388, lng = -74.0018, radius=1, returns=1 )
zipcode = zipcode.to_dict()
zipcode = int(zipcode.get('zipcode'))

In [320]:
zipcode

10011

In [0]:
search = SearchEngine(simple_zipcode=True)

def get_zip_code(latitude, longitude):
  zipcode,  = search.by_coordinates(lat = latitude, lng = longitude, radius=2, returns=1 )
  zipcode = zipcode.to_dict()
  zipcode = int(zipcode.get('zipcode'))
  return zipcode

In [322]:
one_zip_code = get_zip_code(40.5757, -73.9686)
one_zip_code

11235

In [324]:
empty_list = []
for index, row in df.iterrows():
  latitude = row.loc['latitude']
  longitude = row.loc['longitude']
  

KeyboardInterrupt: ignored

In [279]:
len(empty_list)

110

In [0]:
empty_list = []
for index, row in df.iterrows():
  lat = row.loc['latitude']
  long_x = row.loc['longitude']
  empty_list.append((lat, long_x))

In [298]:
df.loc[112]

bathrooms                                 2
bedrooms                                  2
created                 2016-04-11 03:36:06
description                                
display_address                  Ocean Pkwy
latitude                            40.5757
longitude                          -73.9686
price                                  3250
street_address              3111 Ocean Pkwy
interest_level                            1
elevator                                  0
cats_allowed                              0
hardwood_floors                           0
dogs_allowed                              0
doorman                                   1
dishwasher                                0
no_fee                                    0
laundry_in_building                       0
fitness_center                            0
pre-war                                   0
laundry_in_unit                           0
roof_deck                                 0
outdoor_space                   

In [0]:
# THis isnt working cause i need to append it to append at each row.
# This just puts the last value as the entire column

for index, row in df.iterrows():
  latitude = row.loc['latitude']
  longitude = row.loc['longitude']
  df['zipcode'] = get_zip_code(latitude, longitude)

In [327]:
df['zipcode'].value_counts()

10468    48817
Name: zipcode, dtype: int64